In [1]:
import pandas as pd
import os
import re
import numpy as np

# Display all columns when previewing dataframes
from IPython.display import display
pd.options.display.max_columns = None

### Import Top 200 Chart data

In [2]:
# Change path to where spotify_top200.csv is located

# path = '/home/jovyan/work/Documents/Correlation One/spotipy/'
path = r'/Users/minhhoang/Documents/Correlation One/spotipy'
path_input = os.path.join(path, 'input')

In [3]:
# Import check point 1 (dataset includes all songs on charts regardless of collaboration, but does not have API audio features/colalborator names)
data_top200 = pd.read_csv(os.path.join(path_input, 'spotify_top200.csv'))

# Import checkpoint 2 (dataset only includes songs with at least 2 collaborators)
data_top200_reduced = pd.read_csv(os.path.join(path_input, 'spotify_top200_reduced.csv'))\
.drop(columns = ['Unnamed: 0'])

In [ ]:
print(len(data_top200[data_top200.duplicated()]))
# data_top200[data_top200.duplicated()].head()

In [6]:
print(len(data_top200_reduced[data_top200_reduced.duplicated()]))

124211

In [72]:
data_top200_reduced['Year'] = data_top200_reduced['Date'].dt.strftime('%Y')
data_top200_reduced['Year'].unique()

array(['2019', '2020', '2018', '2017'], dtype=object)

In [8]:
data_top200['Date'] = pd.to_datetime(data_top200['Date'])
data_top200_reduced['Date'] = pd.to_datetime(data_top200_reduced['Date'])

In [9]:
# data_top200.groupby(['Country'])[['Position']].describe()
data_top200.loc[(data_top200['Country'].isin(['United States'])) &
                (data_top200['Date'] >= '2020-12-01'),:]\
.groupby(['Date']).describe()

Position                                                      \
              count   mean        std  min    25%    50%     75%    max   
Date                                                                      
2020-12-01    400.0  100.5  57.806609  1.0  50.75  100.5  150.25  200.0   
2020-12-02    400.0  100.5  57.806609  1.0  50.75  100.5  150.25  200.0   
2020-12-03    400.0  100.5  57.806609  1.0  50.75  100.5  150.25  200.0   
2020-12-04    400.0  100.5  57.806609  1.0  50.75  100.5  150.25  200.0   
2020-12-05    400.0  100.5  57.806609  1.0  50.75  100.5  150.25  200.0   
2020-12-06    400.0  100.5  57.806609  1.0  50.75  100.5  150.25  200.0   
2020-12-07    400.0  100.5  57.806609  1.0  50.75  100.5  150.25  200.0   

           Streams                                                            \
             count        mean            std       min        25%       50%   
Date                                                                           
2020-12-01   400.0  382953.155  194109.435538  216120.0  253033.75  313938.5   
2020-12-02   400.0  405067.580  182259.726853  243003.0  283911.00  333107.0   
2020-12-03   400.0  404636.320  180952.683832  242120.0  283171.25  336057.5   
2020-12-04   400.0  429541.285  194266.144861  258476.0  297287.00  346736.0   
2020-12-05   400.0  441886.735  234794.626553  254903.0  288089.75  345251.5   
2020-12-06   400.0  403925.290  219632.993609  231235.0  262511.75  312320.5   
2020-12-07   400.0  385543.085  183254.761892  230386.0  261688.75  312555.0   

                                  
                  75%        max  
Date                              
2020-12-01  430747.25  1180552.0  
2020-12-02  455940.50  1119816.0  
2020-12-03  456117.25  1083677.0  
2020-12-04  487086.25  1184448.0  
2020-12-05  497497.75  1495884.0  
2020-12-06  460554.75  1393655.0  
2020-12-07  439734.50  1145905.0

In [19]:
data_top200_augment = data_top200.drop_duplicates()\
.merge(data_top200_reduced.drop_duplicates(), how='left', 
                            on=['Date', 'Track URL', 'Position', 'Track Name', 
                                'Artist', 'Streams', 'Country'])\
.drop_duplicates()

In [11]:
# print(data_top200.columns)
# print(data_top200_reduced.columns)
print(data_top200_augment.columns)

Index(['Date', 'Track URL', 'Position', 'Track Name', 'Artist', 'Streams',
       'Country', 'Track URI', 'Collaborator URI', 'No. of Collaborators'],
      dtype='object')


In [64]:
print(len(data_top200.index))
print(len(data_top200.drop_duplicates().index))

print(len(data_top200_reduced.index))
print(len(data_top200_reduced.drop_duplicates().index))

14546258
14249658
6581714
6457503


In [20]:
print(len(data_top200_reduced.index))
print(len(data_top200_augment.index))

6581714
14249658


In [22]:
data_top200_augment['Country'] = data_top200_augment['Country'].apply(lambda x: 'Viet Nam' if x == 'Viet Name' else x) 
data_top200_augment['Date'] = pd.to_datetime(data_top200_augment['Date'])

In [24]:
data1 = data_top200_augment.drop_duplicates()
# data1 = data1[data1['Country'] == 'United States']
print(len(data1.index))

14249658


In [27]:
# data1.groupby(['Country'])[['Position']].describe()

In [28]:
data2 = data1.sort_values(by=['Country', 'Artist', 'Date', 'Track Name'], ascending=[True,True,True,True])

### Generate new variables

Calculate a song's earliest appearance on the chart

In [29]:
data3 = data2
data3['Song_min_date'] = data3.groupby(['Country', 'Artist', 'Track Name'])[['Date']].transform('min')

Calculate the number of days between the current chart day and the song's first day on the chart 

In [30]:
data3['Song_days_since_first'] = data3['Date'] - data3['Song_min_date']
data3['Song_days_since_first'] = data3['Song_days_since_first'].apply(lambda x: x.days)

Calculate the actual number of days that the song has appeared on the chart 

In [31]:
data3['Song_days_onchart'] = data3.groupby(['Country', 'Artist', 'Track Name']).cumcount() + 1
# The two previous variables are similar, but may differ if the song temporarily goes off the chart and gets back on at a later date

Identify the first time an artist appears on the Top 200 chart (in a country)
This needs to be run with the complete dataset, not just on songs with collaboration

In [32]:
data3['Artist_min_date'] = data3.groupby(['Country', 'Artist'])[['Date']].transform('min')

Generate a dummy variable that = 1 if this is the first time a song appears

In [33]:
data3['Artist_new_song'] = np.where(data3['Song_days_onchart']==1, 1, 0)

Calculate the number of days elapsed between a song's current appearance and its last appearance
If Date_diff = 1, then the song appeared yesterday and today.

In [34]:
data3['Date_lag'] =  data3.groupby(['Country', 'Artist', 'Track Name'])[['Date']].shift(1)
data3['Date_diff'] = data3['Date'] - data3['Date_lag']
data3['Date_diff'] = data3['Date_diff'].apply(lambda x: x.days)

When Date_diff is > 1 , we assume that the song has temporarily dropped off the chart (hence ending its last consecutive streak),
so we create a binary variable "Song_new_streak" that signals the beginnning of a new streak

In [35]:
data4 = data3
data4['Song_new_streak'] = np.where((data4['Song_days_onchart']==1)|(data4['Date_diff']>1), 1, 0)

For each country-artist-song's consecutive streak, assign a unique id by performing a cumulative sum of the 'Song_new_streak' variable

In [36]:
data4['Song_streak_id'] = data4.groupby(['Country', 'Artist', 'Track Name'])[['Song_new_streak']].transform('cumsum')

For each day within a streak, assign a unique id

In [37]:
data4['Song_consec_day'] = data4.groupby(['Country', 'Artist', 'Track Name', 'Song_streak_id']).cumcount() + 1

Calculate the song's cumulative total number of streams and cumulative average number of streams

In [38]:
data4['Song_cumu_streams'] = data4.groupby(['Country', 'Artist', 'Track Name'])[['Streams']].transform('cumsum')
data4['Song_cumu_mean_streams'] = data4['Song_cumu_streams'] / data4['Song_days_onchart']

Calculate an artist's cumulative number of songs that have appeared on the chart

In [39]:
data5 = data4
data5['Artist_cumsum_songs'] = data5.groupby(['Country', 'Artist'])[['Artist_new_song']].transform('cumsum')
data5['Artist_cumu_songs'] = data5.groupby(['Country', 'Artist', 'Date'])[['Artist_cumsum_songs']].transform('max')

Calculate an artist's cumulative number of days of appearing on the chart (not necessarily consecutively)

In [40]:
artist_days = data5.loc[:, ['Country', 'Artist', 'Date']]\
.sort_values(by=['Country', 'Artist', 'Date'], ascending=[True,True,True]).drop_duplicates()
artist_days['Artist_days_onchart'] = artist_days.groupby(['Country', 'Artist']).cumcount() + 1

artist_days

,Country,Artist,Date,Artist_days_onchart
8773894,Argentina,#TocoParaVos,2017-01-01,1
8806453,Argentina,#TocoParaVos,2017-01-02,2
11463480,Argentina,#TocoParaVos,2017-01-03,3
7710816,Argentina,#TocoParaVos,2017-01-04,4
8751390,Argentina,#TocoParaVos,2017-01-05,5
...,...,...,...,...
2685112,Viet Nam,NaN,2019-08-25,874
2685302,Viet Nam,NaN,2019-08-26,875
2685507,Viet Nam,NaN,2019-08-27,876
2685715,Viet Nam,NaN,2019-08-28,877


Calculate an artist's cumulative total number of streams and cumulative average number of streams

In [41]:
data6 = data5.merge(artist_days, how='left', on=['Country', 'Artist', 'Date'])

data6['Artist_cumsum_streams'] = data6.groupby(['Country', 'Artist'])[['Streams']].transform('cumsum')
data6['Artist_cumu_streams'] = data6.groupby(['Country', 'Artist', 'Date'])[['Artist_cumsum_streams']].transform('max')

# Needs more reflection for this variable - cumulative averages are not accurate because we only observe the song if it's in the top 200
# if the song is not in the top 200, then the cumulative number of streams cannot be updated
# data6['Artist_cumu_mean_streams'] = data6['Artist_cumu_streams'] / data6['Artist_days_onchart']

Generate a unique id for each week and year-week 
(this will be used to merge with other datasets that only update on a weekly basis, e.g. Billboard)

In [42]:
data6['Week_id'] = data6['Date'].dt.strftime('%W')
data6['Year_week_id'] = data6['Date'].dt.strftime('%Y-%W')
data6['Day_week'] = data6['Date'].dt.strftime('%a')

# Create a separate week id to merge with Billboard data, which resets on a Tuesday instead of Monday
data6['Week_id_BB'] = np.where(data6['Day_week'] == 'Mon', 
                               data6['Week_id'].astype(int) - 1, data6['Week_id'].astype(int))

# Calculate the mean number of streams for each week id in a country
data6['Week_mean_streams'] = data6.groupby(['Country', 'Year_week_id'])[['Streams']].transform('mean')

In [43]:
data6.head()

,Date,Track URL,Position,Track Name,Artist,Streams,Country,Track URI,Collaborator URI,No. of Collaborators,Song_min_date,Song_days_since_first,Song_days_onchart,Artist_min_date,Artist_new_song,Date_lag,Date_diff,Song_new_streak,Song_streak_id,Song_consec_day,Song_cumu_streams,Song_cumu_mean_streams,Artist_cumsum_songs,Artist_cumu_songs,Artist_days_onchart,Artist_cumsum_streams,Artist_cumu_streams,Week_id,Year_week_id,Day_week,Week_id_BB,Week_mean_streams
0,2017-01-01,https://open.spotify.com/track/1A3yLIqelu1D7MC...,67,Hasta la luna,#TocoParaVos,35806,Argentina,NaN,NaN,NaN,2017-01-01,0.0,1,2017-01-01,1,NaT,NaN,1,1,1,35806,35806.0,1,3.0,1,35806,71311.0,00,2017-00,Sun,0,39444.360000
1,2017-01-01,https://open.spotify.com/track/7kK1cPOZ9RxZcga...,196,Sólo necesito,#TocoParaVos,14397,Argentina,NaN,NaN,NaN,2017-01-01,0.0,1,2017-01-01,1,NaT,NaN,1,1,1,14397,14397.0,2,3.0,1,50203,71311.0,00,2017-00,Sun,0,39444.360000
2,2017-01-01,https://open.spotify.com/track/6Ol4gn2vFyn4dck...,117,Una historia,#TocoParaVos,21108,Argentina,NaN,NaN,NaN,2017-01-01,0.0,1,2017-01-01,1,NaT,NaN,1,1,1,21108,21108.0,3,3.0,1,71311,71311.0,00,2017-00,Sun,0,39444.360000
3,2017-01-02,https://open.spotify.com/track/1A3yLIqelu1D7MC...,110,Hasta la luna,#TocoParaVos,17585,Argentina,NaN,NaN,NaN,2017-01-01,1.0,2,2017-01-01,0,2017-01-01,1.0,0,1,2,53391,26695.5,3,3.0,2,88896,100234.0,01,2017-01,Mon,0,30851.670714
4,2017-01-02,https://open.spotify.com/track/6Ol4gn2vFyn4dck...,194,Una historia,#TocoParaVos,11338,Argentina,NaN,NaN,NaN,2017-01-01,1.0,2,2017-01-01,0,2017-01-01,1.0,0,1,2,32446,16223.0,3,3.0,2,100234,100234.0,01,2017-01,Mon,0,30851.670714


In [66]:
# data7 = data6[data6['No. of Collaborators'].notna()]
# data7 = data6.merge(data_top200_reduced, how='inner', 
#                     on=['Date', 'Track URL', 'Position', 'Track Name', 
#                         'Artist', 'Streams', 'Country', 
#                         'Track URI', 'Collaborator URI', 'No. of Collaborators'])
data7 = data6[data6['No. of Collaborators'].notna()]
# data7.head()

In [58]:
len(data7[data7['No. of Collaborators'].isna()])

0

In [67]:
len(data7[data7.duplicated()])

0

In [73]:
data7['Year'] = data7['Date'].dt.strftime('%Y').astype(int)
data7['Year'].unique()

/Users/minhhoang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array([2017, 2019, 2020, 2018])

In [68]:
data8 = data7[data7['Country'] == 'United States']

In [51]:
print(len(data_top200.drop_duplicates().index))
print(len(data_top200_reduced.index))
print(len(data_top200_augment.index))

14249658
6581714
14249658
14249658
14249658
14249658
14249658
14249658
14249658
6457503


In [63]:
print(len(data1.index))
print(len(data2.index))
print(len(data3.index))
print(len(data4.index))
print(len(data5.index))
print(len(data6.index))
print(len(data7.index))
print(len(data8.index))

14249658
14249658
14249658
14249658
14249658
14249658
6581714
155545


In [ ]:
# data6.loc[(data6['Artist']=='Drake') &
#           (data6['Country']=='United States') &
#           (data6['Track Name']=='One Dance')
#           ,:]\
# .drop(['Track URL'], axis=1).head()

In [65]:
# data6.loc[(data6['Artist']=='Drake') &
#           (data6['Country']=='United States')
#           ,:]\
# .drop(['Track URL'], axis=1).head(25)

In [ ]:
# data6.to_csv('2020.12.14 top200_clean.csv')

In [70]:
data7.to_csv('2020.12.15 top200_reduced_fe.csv', index=False)

In [69]:
data8.to_csv('2020.12.15 top200_reduced_fe_usa.csv', index=False)

In [ ]:
# data6[data6['Country']=='United States'].to_csv('2020.12.14 top200_usa_clean.csv')